In [9]:
import pickle
import pandas as pd
import numpy as np
import datetime  
import pickle
from math import floor
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDRegressor, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sqlalchemy import create_engine
from sqlalchemy import text
import csv

In [10]:
URI = "dbbikes2.cytgvbje9wgu.us-east-1.rds.amazonaws.com"
PORT = "3306"
DB = "dbbikes2"
USER = "admin"
PASSWORD = "DublinBikes1"   
engine = create_engine("mysql+mysqldb://{}:{}@{}:{}/{}".format(USER, PASSWORD, URI, PORT, DB), echo=True)

In [12]:
"""
# Get the data
availability_data = engine.execute("SELECT * FROM availability2;")

# Create CSV file
filename = "bike_data.csv"

# Open the CSV file in write mode
with open(filename, mode='w', newline='') as file:

    # Create a CSV writer object
    writer = csv.writer(file)

    # Write the header row (optional)
    writer.writerow(['number', 'available_bikes', 'available_bike_stands', 'status', 'datetime'])

    # Write the data rows
    for row in availability_data:
        writer.writerow(row)
"""

2023-04-20 16:13:17,510 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-04-20 16:13:17,512 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-20 16:13:17,609 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-04-20 16:13:17,611 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-20 16:13:17,800 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-04-20 16:13:17,803 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-20 16:13:18,086 INFO sqlalchemy.engine.Engine SELECT * FROM availability2;
2023-04-20 16:13:18,088 INFO sqlalchemy.engine.Engine [raw sql] ()


In [13]:
dfa = pd.read_csv('bike_data.csv')
dfa.head()

,number,available_bikes,available_bike_stands,status,datetime
0,42,8,22,OPEN,1677760001
1,30,2,18,OPEN,1677760001
2,54,14,18,OPEN,1677760001
3,108,18,17,OPEN,1677760001
4,20,22,8,OPEN,1677760001


In [14]:
"""
# Get the data
weather_data = engine.execute("SELECT * FROM weather2;")

# Create CSV file
filename = "weather_data.csv"

# Open the CSV file in write mode
with open(filename, mode='w', newline='') as file:

    # Create a CSV writer object
    writer = csv.writer(file)

    # Write the header row (optional)
    writer.writerow(['main', 'description', 'temp', 'visibility', 'wind_speed', 'wind_direction', 'clouds', 'datetime'])

    # Write the data rows
    for row in weather_data:
        writer.writerow(row)
"""

2023-04-20 16:14:13,348 INFO sqlalchemy.engine.Engine SELECT * FROM weather2;
2023-04-20 16:14:13,350 INFO sqlalchemy.engine.Engine [raw sql] ()


In [15]:
dfw = pd.read_csv('weather_data.csv')
dfw.head()

,main,description,temp,visibility,wind_speed,wind_direction,clouds,datetime
0,Clouds,broken clouds,280.72,10000.0,4.12,70.0,75.0,1677760002
1,Clouds,broken clouds,280.72,10000.0,4.12,70.0,75.0,1677760064
2,Clouds,broken clouds,280.72,10000.0,4.12,70.0,75.0,1677760366
3,Clouds,broken clouds,280.95,10000.0,4.12,70.0,75.0,1677760668
4,Clouds,broken clouds,280.95,10000.0,4.12,70.0,75.0,1677760970


In [16]:
# Round these off so date time is to the nearest hour.
dfw['datetime'] = pd.to_datetime(dfw['datetime'], unit = 's')
dfw['datetime'] = dfw['datetime'].dt.floor('H')
dfw.head()

dfa['datetime'] = pd.to_datetime(dfw['datetime'], unit = 's')
dfa['datetime'] = dfa['datetime'].dt.floor('H')
dfa.head()

,number,available_bikes,available_bike_stands,status,datetime
0,42,8,22,OPEN,2023-03-02 12:00:00
1,30,2,18,OPEN,2023-03-02 12:00:00
2,54,14,18,OPEN,2023-03-02 12:00:00
3,108,18,17,OPEN,2023-03-02 12:00:00
4,20,22,8,OPEN,2023-03-02 12:00:00


In [17]:
# merge the two dataframes on the common column "key"
df = pd.merge(dfa, dfw, on='datetime')
df.head()

,number,available_bikes,available_bike_stands,status,datetime,main,description,temp,visibility,wind_speed,wind_direction,clouds
0,42,8,22,OPEN,2023-03-02 12:00:00,Clouds,broken clouds,280.72,10000.0,4.12,70.0,75.0
1,42,8,22,OPEN,2023-03-02 12:00:00,Clouds,broken clouds,280.72,10000.0,4.12,70.0,75.0
2,42,8,22,OPEN,2023-03-02 12:00:00,Clouds,broken clouds,280.72,10000.0,4.12,70.0,75.0
3,42,8,22,OPEN,2023-03-02 12:00:00,Clouds,broken clouds,280.95,10000.0,4.12,70.0,75.0
4,42,8,22,OPEN,2023-03-02 12:00:00,Clouds,broken clouds,280.95,10000.0,4.12,70.0,75.0


In [18]:
# Add a column for the day of the week (0=Monday, 6=Sunday)
df['dayofweek'] = df['datetime'].dt.dayofweek

# Change day of the week to weekday or weekend
df['weekday_or_weekend'] = df['dayofweek'].apply(lambda x: 'weekday' if x < 5 else 'weekend')
df = df.drop('dayofweek', axis=1)
# Add a column for the hour of the day (0-23)
df['hour'] = df['datetime'].dt.hour
df = df.drop(['datetime', 'status'], axis = 1)

# saving copy to test features on different models
dfc = df
df.head()

,number,available_bikes,available_bike_stands,main,description,temp,visibility,wind_speed,wind_direction,clouds,weekday_or_weekend,hour
0,42,8,22,Clouds,broken clouds,280.72,10000.0,4.12,70.0,75.0,weekday,12
1,42,8,22,Clouds,broken clouds,280.72,10000.0,4.12,70.0,75.0,weekday,12
2,42,8,22,Clouds,broken clouds,280.72,10000.0,4.12,70.0,75.0,weekday,12
3,42,8,22,Clouds,broken clouds,280.95,10000.0,4.12,70.0,75.0,weekday,12
4,42,8,22,Clouds,broken clouds,280.95,10000.0,4.12,70.0,75.0,weekday,12


In [19]:
df = pd.get_dummies(df, columns=['main', 'description', 'weekday_or_weekend'])
df.head()

,number,available_bikes,available_bike_stands,temp,visibility,wind_speed,wind_direction,clouds,hour,main_Clear,...,description_mist,description_moderate rain,description_overcast clouds,description_ragged shower rain,description_scattered clouds,description_shower rain,description_sleet,description_snow,weekday_or_weekend_weekday,weekday_or_weekend_weekend
0,42,8,22,280.72,10000.0,4.12,70.0,75.0,12,0,...,0,0,0,0,0,0,0,0,1,0
1,42,8,22,280.72,10000.0,4.12,70.0,75.0,12,0,...,0,0,0,0,0,0,0,0,1,0
2,42,8,22,280.72,10000.0,4.12,70.0,75.0,12,0,...,0,0,0,0,0,0,0,0,1,0
3,42,8,22,280.95,10000.0,4.12,70.0,75.0,12,0,...,0,0,0,0,0,0,0,0,1,0
4,42,8,22,280.95,10000.0,4.12,70.0,75.0,12,0,...,0,0,0,0,0,0,0,0,1,0


In [20]:
# Split data into training and testing data
X = df.drop(['available_bikes', 'available_bike_stands'], axis = 1)
y = df['available_bikes']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

# Logic Regression Model

In [21]:
# create and fit the model
model = LogisticRegression()
model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# Calculate root mean squared error (RMSE)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print("Root Mean Squared Error (RMSE):", rmse)

# Calculate R-squared score
r2 = r2_score(y_test, y_pred)
print("R-squared score:", r2)

print("Mean absolute error:", mean_absolute_error(y_test, y_pred))

Root Mean Squared Error (RMSE): 14.408675588780355
R-squared score: -1.3220537619517976
Mean absolute error: 10.917717109013745


/Users/conormcelduff/opt/anaconda3/envs/comp30830/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Linear Regression Model

In [22]:
# create and fit the model
model = LinearRegression()
model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# Calculate root mean squared error (RMSE)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("Root Mean Squared Error (RMSE):", rmse)

# Calculate R-squared score
r2 = r2_score(y_test, y_pred)
print("R-squared score:", r2)

# Calculate mean square error
print("Mean absolute error:", mean_absolute_error(y_test, y_pred))

Root Mean Squared Error (RMSE): 9.447477431003593
R-squared score: 0.0017119551449467085
Mean absolute error: 7.709657295957276


# Random Forest Regressor

In [23]:
# Create a random forest regressor with 100 trees
model = RandomForestRegressor(n_estimators = 50, random_state = 36)
model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# Calculate root mean squared error (RMSE)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("Root Mean Squared Error (RMSE):", rmse)

# Calculate R-squared score
r2 = r2_score(y_test, y_pred)
print("R-squared score:", r2)

# Calculate mean square error
print("Mean absolute error:", mean_absolute_error(y_test, y_pred))
print(y_pred)

Root Mean Squared Error (RMSE): 1.3301623455312752
R-squared score: 0.9802105628070497
Mean absolute error: 0.4112944828023057
[ 0.    3.38  8.06 ...  6.    5.   11.92]


# SGD Regressor 

In [24]:
# create and fit the model
model = SGDRegressor()
model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# Calculate root mean squared error (RMSE)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("Root Mean Squared Error (RMSE):", rmse)

# Calculate R-squared score
r2 = r2_score(y_test, y_pred)
print("R-squared score:", r2)

# Calculate mean square error
print("Mean absolute error:", mean_absolute_error(y_test, y_pred))

Root Mean Squared Error (RMSE): 1385316191972404.8
R-squared score: -2.146455873688554e+28
Mean absolute error: 1362143661667094.0


# Removing Main Weather and Description

In [25]:
# Taking copy dfc I save earlier 
df = dfc
# Will test dropping main weather description
df = df.drop(['main', 'description'], axis = 1)
df = pd.get_dummies(df, columns=['weekday_or_weekend'])
X = df.drop(['available_bikes', 'available_bike_stands'], axis = 1)
y = df['available_bikes']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

# create and fit the model
model = RandomForestRegressor(n_estimators = 50, random_state = 36)
model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# Calculate root mean squared error (RMSE)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("Root Mean Squared Error (RMSE):", rmse)

# Calculate R-squared score
r2 = r2_score(y_test, y_pred)
print("R-squared score:", r2)

# Calculate mean square error
print("Mean absolute error:", mean_absolute_error(y_test, y_pred))

Root Mean Squared Error (RMSE): 1.2775203936597739
R-squared score: 0.9817459251337155
Mean absolute error: 0.3854097041869893


In [26]:
# Taking copy dfc I save earlier 
df = dfc
# Will test dropping main weather description
df = df.drop(['main', 'description', 'temp'], axis = 1)
df = pd.get_dummies(df, columns=['weekday_or_weekend'])
X = df.drop(['available_bikes', 'available_bike_stands'], axis = 1)
y = df['available_bikes']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

# create and fit the model
model = RandomForestRegressor(n_estimators = 50, random_state = 36)
model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# Calculate root mean squared error (RMSE)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("Root Mean Squared Error (RMSE):", rmse)

# Calculate R-squared score
r2 = r2_score(y_test, y_pred)
print("R-squared score:", r2)

# Calculate mean square error
print("Mean absolute error:", mean_absolute_error(y_test, y_pred))

Root Mean Squared Error (RMSE): 1.5964664900238066
R-squared score: 0.9714935092013245
Mean absolute error: 0.4726872315035245


In [27]:
# Taking copy dfc I save earlier 
df = dfc
# Will test dropping main weather description
df = df.drop(['main', 'description', 'visibility'], axis = 1)
df = pd.get_dummies(df, columns=['weekday_or_weekend'])
X = df.drop(['available_bikes', 'available_bike_stands'], axis = 1)
y = df['available_bikes']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

# create and fit the model
model = RandomForestRegressor(n_estimators = 50, random_state = 36)
model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# Calculate root mean squared error (RMSE)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("Root Mean Squared Error (RMSE):", rmse)

# Calculate R-squared score
r2 = r2_score(y_test, y_pred)
print("R-squared score:", r2)

# Calculate mean square error
print("Mean absolute error:", mean_absolute_error(y_test, y_pred))

Root Mean Squared Error (RMSE): 1.27996171658931
R-squared score: 0.9816760919280525
Mean absolute error: 0.386691032706235


In [28]:
# Taking copy dfc I save earlier 
df = dfc
# Will test dropping main weather description
df = df.drop(['main', 'description', 'visibility', 'wind_speed'], axis = 1)
df = pd.get_dummies(df, columns=['weekday_or_weekend'])
X = df.drop(['available_bikes', 'available_bike_stands'], axis = 1)
y = df['available_bikes']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

# create and fit the model
model = RandomForestRegressor(n_estimators = 50, random_state = 36)
model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# Calculate root mean squared error (RMSE)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("Root Mean Squared Error (RMSE):", rmse)

# Calculate R-squared score
r2 = r2_score(y_test, y_pred)
print("R-squared score:", r2)

# Calculate mean square error
print("Mean absolute error:", mean_absolute_error(y_test, y_pred))

Root Mean Squared Error (RMSE): 1.248960601154496
R-squared score: 0.9825529654438871
Mean absolute error: 0.36690756861432094


In [29]:
# Taking copy dfc I save earlier 
df = dfc
# Will test dropping main weather description
df = df.drop(['main', 'description', 'visibility', 'wind_direction'], axis = 1)
df = pd.get_dummies(df, columns=['weekday_or_weekend'])
X = df.drop(['available_bikes', 'available_bike_stands'], axis = 1)
y = df['available_bikes']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

# create and fit the model
model = RandomForestRegressor(n_estimators = 50, random_state = 36)
model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# Calculate root mean squared error (RMSE)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("Root Mean Squared Error (RMSE):", rmse)

# Calculate R-squared score
r2 = r2_score(y_test, y_pred)
print("R-squared score:", r2)

# Calculate mean square error
print("Mean absolute error:", mean_absolute_error(y_test, y_pred))

Root Mean Squared Error (RMSE): 1.4577789395599712
R-squared score: 0.97623118735419
Mean absolute error: 0.47477004456907224


In [30]:
# Taking copy dfc I save earlier 
df = dfc
# Will test dropping main weather description
df = df.drop(['main', 'description', 'visibility', 'clouds'], axis = 1)
df = pd.get_dummies(df, columns=['weekday_or_weekend'])
X = df.drop(['available_bikes', 'available_bike_stands'], axis = 1)
y = df['available_bikes']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

# create and fit the model
model = RandomForestRegressor(n_estimators = 50, random_state = 36)
model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# Calculate root mean squared error (RMSE)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("Root Mean Squared Error (RMSE):", rmse)

# Calculate R-squared score
r2 = r2_score(y_test, y_pred)
print("R-squared score:", r2)

# Calculate mean square error
print("Mean absolute error:", mean_absolute_error(y_test, y_pred))

Root Mean Squared Error (RMSE): 1.3028078844272073
R-squared score: 0.9810161234800916
Mean absolute error: 0.39792601507569514


Our features being used in final model are:

* Station Number
* Temperature
* Wind Speed
* Wind direction
* Clouds
* Hour
* Weekday or Weekend

In [31]:
# Taking copy dfc I save earlier 
df = dfc
# Will test dropping main weather description
df = df.drop(['main', 'description', 'visibility'], axis = 1)
df = pd.get_dummies(df, columns=['weekday_or_weekend'])
X = df.drop(['available_bikes', 'available_bike_stands'], axis = 1)
y = df['available_bikes']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

model = RandomForestRegressor(n_estimators = 50, random_state = 36)
model.fit(X_train, y_train)
# Serialize model object into a file called model.pkl on disk using pickle
with open('model.pkl', 'wb') as handle:
    pickle.dump(model, handle, pickle.HIGHEST_PROTOCOL)
# pickle.HIGHEST_PROTOCOL using the highest available protocol
# (we used wb to open file as binary and use a higher pickling protocol)

In [32]:
X.head()

,number,temp,wind_speed,wind_direction,clouds,hour,weekday_or_weekend_weekday,weekday_or_weekend_weekend
0,42,280.72,4.12,70.0,75.0,12,1,0
1,42,280.72,4.12,70.0,75.0,12,1,0
2,42,280.72,4.12,70.0,75.0,12,1,0
3,42,280.95,4.12,70.0,75.0,12,1,0
4,42,280.95,4.12,70.0,75.0,12,1,0
